# **🔥 Smoke & Fire Detection – README**

### Random Forest + Cross Validation

Bem-vindo(a) ao projeto! 🚀
Aqui, brinquei de detetive do fogo usando dados ambientais coletados por sensores IoT.
A missão é simples:
👉 Treinar um modelo de Machine Learning que consiga prever se existe ou não um incêndio, usando variáveis do ambiente como temperatura, umidade, gases e partículas suspensas no ar.

📊 Sobre a Base de Dados - smoke_detection_.csv

O dataset é quase como uma central de monitoramento ambiental em miniatura. Ele traz 62 mil registros com diversas variáveis medidas em tempo real:

UTC → tempo em segundos (cronômetro da coleta ⏱️)

Temperature [C] → temperatura do ar 🌡️

Humidity [%] → umidade do ar 💧

TVOC [ppb] → compostos orgânicos voláteis (basicamente, cheiros suspeitos 👃)

eCO2 [ppm] → concentração equivalente de dióxido de carbono 🌫️

Raw H2 → nível bruto de hidrogênio

Raw Ethanol → nível bruto de etanol (sim, até isso o sensor pega 🍷😅)

Pressure [hPa] → pressão atmosférica 🌍

PM1.0 / PM2.5 → partículas sólidas no ar, tipo fumaça invisível 😶‍🌫️

NC0.5 / NC1.0 / NC2.5 → contagem numérica de partículas por tamanho

CNT → contador de amostras (basicamente, quantos registros o sensor já pegou 📈)

E, claro, a cereja do bolo:

🎯 Fire_Alarm → nossa variável alvo!

1 = incêndio detectado 🔥

0 = tudo sob controle ✅

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 1 - Base de dados, tipos de dados e se há presença de dados faltantes ou nulos.

# **Pré Tratamento:**

- Dropar coluna irrelevante.

In [ ]:
# Abrir Df
df = pd.read_csv('smoke_detection_.csv')

# Exibe as primeiras 5 linhas do dataset
df.head()

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [ ]:
# Mostrar informações gerais do dataframe
print("\nInformações do dataframe:")
df.info()


Informações do dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature[C]  62630 non-null  float64
 1   Humidity[%]     62630 non-null  float64
 2   TVOC[ppb]       62630 non-null  int64  
 3   eCO2[ppm]       62630 non-null  int64  
 4   Raw H2          62630 non-null  int64  
 5   Raw Ethanol     62630 non-null  int64  
 6   Pressure[hPa]   62630 non-null  float64
 7   PM1.0           62630 non-null  float64
 8   PM2.5           62630 non-null  float64
 9   NC0.5           62630 non-null  float64
 10  NC1.0           62630 non-null  float64
 11  NC2.5           62630 non-null  float64
 12  CNT             62630 non-null  int64  
 13  Fire_Alarm      62630 non-null  int64  
 14  hour            62630 non-null  int32  
 15  weekday         62630 non-null  int32  
 16  month           62630 non-null  int32  
dtypes: f

In [ ]:
# Remove a coluna 'Unnamed: 0'
df = df.drop(columns=["Unnamed: 0"])

# Conferir se sumiu
print(df.head())

          UTC  Temperature[C]  Humidity[%]  TVOC[ppb]  eCO2[ppm]  Raw H2  \
0  1654733331          20.000        57.36          0        400   12306   
1  1654733332          20.015        56.67          0        400   12345   
2  1654733333          20.029        55.96          0        400   12374   
3  1654733334          20.044        55.28          0        400   12390   
4  1654733335          20.059        54.69          0        400   12403   

   Raw Ethanol  Pressure[hPa]  PM1.0  PM2.5  NC0.5  NC1.0  NC2.5  CNT  \
0        18520        939.735    0.0    0.0    0.0    0.0    0.0    0   
1        18651        939.744    0.0    0.0    0.0    0.0    0.0    1   
2        18764        939.738    0.0    0.0    0.0    0.0    0.0    2   
3        18849        939.736    0.0    0.0    0.0    0.0    0.0    3   
4        18921        939.744    0.0    0.0    0.0    0.0    0.0    4   

   Fire Alarm  
0           0  
1           0  
2           0  
3           0  
4           0  


- Tipos de Dados:

In [ ]:
# Verificar tipos de dados de cada coluna
print(df.dtypes)

UTC                 int64
Temperature[C]    float64
Humidity[%]       float64
TVOC[ppb]           int64
eCO2[ppm]           int64
Raw H2              int64
Raw Ethanol         int64
Pressure[hPa]     float64
PM1.0             float64
PM2.5             float64
NC0.5             float64
NC1.0             float64
NC2.5             float64
CNT                 int64
Fire Alarm          int64
dtype: object


**Insigth:** Notei que UTC está em segundos... Contabilizando valores muitos altos acumulados.


Vou converter os segundos features significantes para o estudo. Como minha intenção nesse estudo é usar o Randon Forest e XGBoost, esses modelos entendem melhor essas features que vou criar.

In [ ]:
# Converter UTC (segundos) para datetime
df['UTC'] = pd.to_datetime(df['UTC'], unit='s')

# Agora sim dá pra criar as features derivadas
df['hour'] = df['UTC'].dt.hour
df['weekday'] = df['UTC'].dt.weekday
df['month'] = df['UTC'].dt.month


Dropar a coluna UTC original para não gerar redundância.

In [ ]:
df = df.drop(columns=['UTC'])

- Renomear a coluna Fire Alarm.

In [ ]:
df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

# Conferir se funcionou
print(df.columns)


Index(['Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2',
       'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5', 'NC0.5', 'NC1.0',
       'NC2.5', 'CNT', 'Fire_Alarm', 'hour', 'weekday', 'month'],
      dtype='object')


- Visualização.

In [ ]:
df.head(10)

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire_Alarm,hour,weekday,month
0,20.000,57.36,0,400,12306,18520,939.735,0.0,0.00,0.0,0.000,0.00,0,0,0,3,6
1,20.015,56.67,0,400,12345,18651,939.744,0.0,0.00,0.0,0.000,0.00,1,0,0,3,6
2,20.029,55.96,0,400,12374,18764,939.738,0.0,0.00,0.0,0.000,0.00,2,0,0,3,6
3,20.044,55.28,0,400,12390,18849,939.736,0.0,0.00,0.0,0.000,0.00,3,0,0,3,6
4,20.059,54.69,0,400,12403,18921,939.744,0.0,0.00,0.0,0.000,0.00,4,0,0,3,6
5,20.073,54.12,0,400,12419,18998,939.725,0.0,0.00,0.0,0.000,0.00,5,0,0,3,6
6,20.088,53.61,0,400,12432,19058,939.738,0.0,0.00,0.0,0.000,0.00,6,0,0,3,6
7,20.103,53.20,0,400,12439,19114,939.758,0.0,0.00,0.0,0.000,0.00,7,0,0,3,6
8,20.117,52.81,0,400,12448,19155,939.758,0.0,0.00,0.0,0.000,0.00,8,0,0,3,6
9,20.132,52.46,0,400,12453,19195,939.756,0.9,3.78,0.0,4.369,2.78,9,0,0,3,6


# 2 - Escolha de Modelo de Machiner Learning.



Conforme citado anteriormente, optei pelo Random Forest para este ML.
Como os dados são de natureza sensorial e as relações entre as variáveis não são evidentes, o Random Forest é apropriado para identificar padrões complexos e interações entre as variáveis.

# 3 - Separe a base em Y e X e já rode a instância do modelo que você utilizará.

In [ ]:
# Variável alvo
Y = df['Fire_Alarm']

# Variáveis de entrada
X = df.drop(columns=['Fire_Alarm'])

**Insigth:** Detecção de fumaça (possivelmente dados desequilibrados e relativamente pequenos), 70/30 é uma boa escolha.

Tamanho do DF [62.630 linhas (entries)]: 1.000 – 100.000 linhas
Médio
Treinamento rápido, CV funciona bem, geralmente suficiente para Random Forest.

- Machine Learning (RandomForest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Divisão treino/teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

# Instanciando o modelo
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Treinar o modelo
rf_model.fit(X_train, Y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

- Treinamento.

In [ ]:
Y_pred = rf_model.predict(X_test)

In [ ]:
# Criar um DataFrame comparando valores reais e preditos
import pandas as pd

results = pd.DataFrame({'Real': Y_test, 'Predito': Y_pred})
results.head(10)


,Real,Predito
40666,1,1
8769,1,1
52541,0,0
1673,0,0
15789,1,1
13811,1,1
56930,0,0
19328,1,1
56830,0,0
699,0,0


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Acurácia
acc = accuracy_score(Y_test, Y_pred)
print(f"Acurácia: {acc:.4f}")

# Relatório detalhado
print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Matriz de confusão
print("\nMatriz de Confusão:")
print(confusion_matrix(Y_test, Y_pred))


Acurácia: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5362
           1       1.00      1.00      1.00     13427

    accuracy                           1.00     18789
   macro avg       1.00      1.00      1.00     18789
weighted avg       1.00      1.00      1.00     18789


Matriz de Confusão:
[[ 5362     0]
 [    0 13427]]


**Insigth:** Assertividade MUITO ALTA!

**Possíveis pontos de atenção!**

**Overfitting:**

Com Random Forest, é raro ter 100% de acerto em dados reais.

O modelo está decorando padrões específicos do dataset, especialmente se as features derivadas (hour, weekday, month) e sensores têm correlação muito direta com o target.

**Dataset muito previsível:**

Talvez os sensores já indiquem claramente quando há fogo, então o problema é relativamente “fácil” para o modelo.

**Confirmação com CV:**

Mesmo assim, é uma boa prática rodar cross-validation para garantir que essa performance não é só um acaso da divisão treino/teste.

# 4 - Número de Folds e modelo com a validação cruzada.

# **K=5**

Equilíbrio entre precisão e custo computacional

Quanto maior o k, mais folds, mais treinos o modelo faz → mais preciso, mas mais demorado.

Quanto menor o k, menos folds → mais rápido, mas a média das acurácias pode ser menos confiável.

Prática comum

k = 5 ou k = 10 são os valores mais usados.

Para datasets médios, 5 folds já fornecem uma boa estimativa da performance sem gastar muito tempo de processamento.


Com 5 folds, cada fold tem cerca de 20% do dataset como teste e 80% como treino.

Isso mantém o teste grande o suficiente para avaliar o modelo e o treino grande o suficiente para aprender padrões robustos.

In [ ]:
k = 5


In [ ]:
# Instanciar o modelo (mesma configuração usada antes)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Aplicar cross-validation
scores = cross_val_score(rf_model, X, Y, cv=k, scoring='accuracy')

# Mostrar resultados
print(f"Acurácias em cada fold: {scores}")
print(f"Acurácia média: {scores.mean():.4f}")


Acurácias em cada fold: [1.         0.99736548 0.9823567  1.         0.91018681]
Acurácia média: 0.9780


**Insigth:**
Excelente resultado!
Mais confiável do que o treino/teste único, porque considera diferentes subconjuntos do dataset

# 5 - Avaliação.

A validação cruzada evita que o desempenho do modelo seja influenciado por uma única divisão dos dados, que poderia levar a resultados muito otimistas ou pessimistas.
Ao avaliar várias divisões (folds), conseguimos ter uma visão mais confiável e robusta de como o modelo se comporta em diferentes subconjuntos do dataset.
Além disso, é possível identificar quais folds apresentam maior dificuldade, como o último fold que teve cerca de 91% de acerto, permitindo entender melhor a variabilidade e os limites do modelo.
A média das acurácias de todos os folds, nesse caso 97,8%, representa uma estimativa final mais realista da performance do Random Forest neste problema.